In [1]:
# preprocess Links

In [74]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [140]:
import requests
from bs4 import BeautifulSoup

def get_clean_data(url):
    try :
        content = requests.get(url)
        content = content.text
        
    except:
        content = ''
        
    bs = BeautifulSoup(content,'html.parser')
    texts = bs.findAll(['title','p','strong','li',re.compile('^h[1-6]$')])
    return texts


In [136]:
import re
def remove_tags(content):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', content)
    
    return cleantext

In [137]:
def remove_punctuation(content):
    content = re.sub("_", "", content)
    content = re.sub("[^\w\s]", "", content)
    content = re.sub(' +', ' ', content)

    return content

In [78]:
data = get_clean_data('https://www.toolshero.com/toolsheroes/elon-musk/')
print(remove_punctuation(remove_tags(str(data))))

Elon Musk biography quotes publications and books toolshero Follow us Facebook Twitter Instagram YouTube Pinterest More Toolkit Change Management
Communication Skills
Creativity
Decision Making
Financial Management
Human Resources HR
Information Technology IT
Innovation
Leadership
Management
Marketing
Personal Development
Project Management
Problem Solving
Psychology
Quality Management
Strategy
Toolsheroes

 Change Management Communication Skills Creativity Decision Making Financial Management Human Resources HR Information Technology IT Innovation Leadership Management Marketing Personal Development Project Management Problem Solving Psychology Quality Management Strategy Toolsheroes Templates Find Jobs Find Courses Find Tests More Menu Search Elon Musk Elon Musk is a CanadianAmerican entrepreneur He is the founder of SpaceX Tesla and Zip2 among others He was also one of the people responsible for the success of PayPal
Currently he serves as CEO and Chief Designer CDO of SpaceX CEO an

In [92]:
def convert_to_vector(content,vocabulary):
    vector_dict = dict.fromkeys(vocabulary,0)
    doc = nlp(content)

    for token in doc:
        if token.text.lower() in vector_dict:
            vector_dict[token.text.lower()] += 1
    
    return vector_dict.values()

    

In [93]:
def create_vocabulary(list_of_queries):
    vocabulary = set()
    for query in list_of_queries:
        words = query.lower().split()
        vocabulary.update(words)
    
    return vocabulary
        

In [82]:
import pandas as pd

In [86]:
vocab = create_vocabulary(list(set(pd.read_csv('Queries.csv')['Query'])))

In [87]:
vocab

{'11/20/2019',
 'a',
 'abraham',
 'aj',
 'ali',
 'bee',
 'bill',
 'by',
 'celtics',
 'crowe',
 'elon',
 "gandhi's",
 'gettysburg',
 'gladiator',
 'goodell',
 "green's",
 'health',
 'inauguration',
 'jfk',
 'like',
 'lincoln',
 'mohammad',
 'musk',
 'obama',
 'on',
 'president',
 'quotes',
 'race',
 'roger',
 'russell',
 'simmons',
 'space',
 'sting',
 'truth',
 'views'}

In [280]:
pd.read_csv('Queries.csv')

,Query,URL,Ranking
0,Abraham Lincoln Gettysburg,http://www.abrahamlincolnonline.org/lincoln/sp...,1
1,Abraham Lincoln Gettysburg,https://www.history.com/topics/american-civil-...,4
2,Abraham Lincoln Gettysburg,https://rmc.library.cornell.edu/gettysburg/goo...,4
3,Abraham Lincoln Gettysburg,https://www.historynet.com/gettysburg-address-...,3
4,Abraham Lincoln Gettysburg,https://www.independent.co.uk/news/world/ameri...,4
...,...,...,...
190,quotes by Elon Musk,https://www.brainyquote.com/authors/elon-musk-...,4
191,quotes by Elon Musk,https://www.amazon.com/Elon-Musk-Entrepreneur-...,2
192,quotes by Elon Musk,https://highexistence.com/50-elon-musk-quotes/,4
193,quotes by Elon Musk,https://www.youthkiawaaz.com/2019/08/some-quot...,3


In [293]:
def make_data_frame(document, vocab):
    """
    Converts data into a pandas data frame with bag of words features instead of just a text feature.
    -----
    | Id , text, label| -> |Id, token_0 ... token_n , Label|
    """
    temp_df = []
    
    for _, row in document.iterrows():
        query = row['Query']
        url = row['URL']
        rank = row['Ranking']
        
        text = (remove_punctuation(remove_tags(str(get_clean_data(url)))))
        row = [query, url] + list(convert_to_vector(text, vocab)) +[rank]
        temp_df.append(row)
    
    df = pd.DataFrame(temp_df, columns=['Query', 'Url'] + list(vocab) +['Human Rank'])
    return df

In [294]:
doc =pd.read_csv('Queries.csv')[['Query','URL','Ranking']]

In [295]:
doc

,Query,URL,Ranking
0,Abraham Lincoln Gettysburg,http://www.abrahamlincolnonline.org/lincoln/sp...,1
1,Abraham Lincoln Gettysburg,https://www.history.com/topics/american-civil-...,4
2,Abraham Lincoln Gettysburg,https://rmc.library.cornell.edu/gettysburg/goo...,4
3,Abraham Lincoln Gettysburg,https://www.historynet.com/gettysburg-address-...,3
4,Abraham Lincoln Gettysburg,https://www.independent.co.uk/news/world/ameri...,4
...,...,...,...
190,quotes by Elon Musk,https://www.brainyquote.com/authors/elon-musk-...,4
191,quotes by Elon Musk,https://www.amazon.com/Elon-Musk-Entrepreneur-...,2
192,quotes by Elon Musk,https://highexistence.com/50-elon-musk-quotes/,4
193,quotes by Elon Musk,https://www.youthkiawaaz.com/2019/08/some-quot...,3


In [296]:
vocab_df = make_data_frame(doc[:3], vocab)

In [297]:
vocab_df.head()

,Query,Url,aj,russell,views,musk,health,jfk,inauguration,on,...,like,a,celtics,11/20/2019,simmons,bee,goodell,gandhi's,crowe,Human Rank
0,Abraham Lincoln Gettysburg,http://www.abrahamlincolnonline.org/lincoln/sp...,0,0,0,0,0,0,0,129,...,0,521,0,0,0,0,0,0,0,1
1,Abraham Lincoln Gettysburg,https://www.history.com/topics/american-civil-...,0,0,0,0,0,0,0,15,...,0,28,0,0,0,0,0,0,0,4
2,Abraham Lincoln Gettysburg,https://rmc.library.cornell.edu/gettysburg/goo...,0,0,0,0,0,0,0,4,...,0,8,0,0,0,0,0,0,0,4


In [169]:
vocab_df.iloc[-1]

Query           Abraham Lincoln Gettysburg
aj                                       0
russell                                  0
views                                    0
musk                                     0
health                                   0
jfk                                      0
inauguration                             0
on                                       4
elon                                     0
abraham                                  0
gettysburg                               3
race                                     0
space                                    0
by                                       1
obama                                    0
truth                                    0
quotes                                   0
mohammad                                 0
green's                                  0
lincoln                                  1
president                                1
sting                                    0
roger      

In [301]:
import numpy as np
def get_liklihood_score(row, D):
    tot = 0
    for term in row['Query'].lower().split():
#         print(row.keys())
        tot += np.log((row[term] + 1) /(D+ len(row.keys())-1))
    return tot

In [302]:
vocab_df['score'] = [get_liklihood_score(row,len(vocab_df)) for ind, row in vocab_df.iterrows()]

In [303]:
list(vocab_df.sort_values(by=['score'], ascending=False)['score'])

[4.7577796787965205, -3.0569430045988875, -8.987196820661973]

In [304]:
vocab_df['score'].quantile(0.8)

1.631890605438358

In [305]:
def score_fix(row):
    score = row['score']
    if vocab_df['score'].quantile(.2) > score:
        return 0
    elif vocab_df['score'].quantile(.4) > score:
        return 1
    elif vocab_df['score'].quantile(.6) > score:
        return 2
    elif vocab_df['score'].quantile(.8) > score:
        return 3
    else:
        return 4

In [306]:
vocab_df["fitted_score"]=vocab_df.apply(score_fix, axis=1)

In [307]:
vocab_df.head()

,Query,Url,aj,russell,views,musk,health,jfk,inauguration,on,...,celtics,11/20/2019,simmons,bee,goodell,gandhi's,crowe,Human Rank,score,fitted_score
0,Abraham Lincoln Gettysburg,http://www.abrahamlincolnonline.org/lincoln/sp...,0,0,0,0,0,0,0,129,...,0,0,0,0,0,0,0,1,4.757780,4
1,Abraham Lincoln Gettysburg,https://www.history.com/topics/american-civil-...,0,0,0,0,0,0,0,15,...,0,0,0,0,0,0,0,4,-3.056943,2
2,Abraham Lincoln Gettysburg,https://rmc.library.cornell.edu/gettysburg/goo...,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,4,-8.987197,0


In [308]:
vocab_df[vocab_df['Query'] == 'Abraham Lincoln Gettysburg'].sort_values(by=['score'], ascending=False)

,Query,Url,aj,russell,views,musk,health,jfk,inauguration,on,...,celtics,11/20/2019,simmons,bee,goodell,gandhi's,crowe,Human Rank,score,fitted_score
0,Abraham Lincoln Gettysburg,http://www.abrahamlincolnonline.org/lincoln/sp...,0,0,0,0,0,0,0,129,...,0,0,0,0,0,0,0,1,4.757780,4
1,Abraham Lincoln Gettysburg,https://www.history.com/topics/american-civil-...,0,0,0,0,0,0,0,15,...,0,0,0,0,0,0,0,4,-3.056943,2
2,Abraham Lincoln Gettysburg,https://rmc.library.cornell.edu/gettysburg/goo...,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,4,-8.987197,0
